# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [191]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from math import *


In [192]:
#leitura do arquivo excel
budweiser=pd.read_excel("tweets_budweiser.xlsx")
#print(budweiser)


  

In [193]:
#Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc.

budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('@', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace("'", '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('"', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('.', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('(', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace(')', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('@', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('$', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('*', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace(':', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace(';', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('-', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('_', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('%', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('/', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('!', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('?', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('[', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace(']', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('{', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('}', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('+', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('>', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('<', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('^', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('~', '')  
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('=', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('#', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('+', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('•', '')
#budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('...', '')





#Propor outras limpezas/transformações que não afetem a qualidade da informação.

budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('eu', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('rt', '')
budweiser["Treinamento"]=budweiser["Treinamento"].str.replace('https://...', '')


print(budweiser)                                                         

                                           Treinamento  Classificação
0    soutomc anurbanshop budweiserbr chinaskioficia...              0
1    essa budweiser me fud ontem, dor de cabeça tá ...              1
2     david3rick crlh budweiser com red bull é um c...              1
3     promodojaca jogo do mengão amanhã , vem ver a...              1
4     hailaffc budweiser é a melhor cerveja e nada ...              1
5    1  welcome bud quem for com camisa do eagles o...              1
6       marianycostaa budweiser é a melhor cerveja tnc              1
7    luizzalb prefiro budweiser quaa e quinta e o r...              1
8     niallhbra2 📌| niall se apresenta hoje no budw...              0
9    rolezinho q  dou valor 2 budweiser no mundial,...              1
10   mais em compensação bebemos dois packs de budw...              1
11                  não tem stella vai desce budweiser              1
12                    budweiser é a melhor cerveja tnc              1
13   jogo do mengão 

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [194]:
relevante=0
irrelevante=0
for i in budweiser['Classificação']:
    if i==0:
        irrelevante+=1
    elif i==1:
        relevante+=1
print(relevante)
print(irrelevante)
        
#Probabilidade de ser relevante

prob_relevante=relevante/300

prob_irrelevante=irrelevante/300
 
print(prob_irrelevante*100,prob_relevante*100)

194
106
35.333333333333336 64.66666666666666


In [195]:
relevantes_counts=budweiser["Treinamento"][budweiser.Classificação==1]
relevantes_counts

palavras_rel=[]
for r in relevantes_counts:
    for e in r.split():
        palavras_rel.append(e)
print(palavras_rel)

irrelevantes_counts=budweiser['Treinamento'][budweiser.Classificação==0]
palavras_irr=[]
for i in irrelevantes_counts:
    for e in i.split():
        palavras_irr.append(e)
print(palavras_irr)



['essa', 'budweiser', 'me', 'fud', 'ontem,', 'dor', 'de', 'cabeça', 'tá', 'tendo', 'ms', 'amigos', '🤦🏿\u200d♂😂😂😂😂', 'david3rick', 'crlh', 'budweiser', 'com', 'red', 'bull', 'é', 'um', 'caminho', 'sem', 'volta', 'deixa', 'mal', '\U0001f92d🍻🕺🏼', 'promodojaca', 'jogo', 'do', 'mengão', 'amanhã', ',', 'vem', 'ver', 'aquele', 'jogo', 'tomando', 'uma', 'budweiser', 'gelada', ',', 'mega', 'promoção', 'a', 'pai', 'de', 'amanhã', 'no', 'bar', 'do', 'leo', 'n…', 'hailaffc', 'budweiser', 'é', 'a', 'melhor', 'cerveja', 'e', 'nada', 'mais', 'impoa', 'a', 'não', 'ser', 'minha', 'opinião', '1', 'welcome', 'bud', 'quem', 'for', 'com', 'camisa', 'do', 'eagles', 'ou', 'do', 'falcons', 'ganha', 'uma', 'budweiser', 'long', 'neck', '2', 'preço', 'da', 'pint', 'reduzido', 'pint', 'lager', 'schornstein', 'por', '11,90', 'mais', 'barato', 'do', 'que', 'long', 'neck', '3', 'balde', 'promocional', 'balde', 'de', 'budweiser', 'compre', '5', 'leve', '6', 'a', 'noite', 'toda', 'marianycostaa', 'budweiser', 'é', 'a'

In [196]:
dic_rel = {}
dic_ambas={}
for r in palavras_rel:
    dic_rel[r]=palavras_rel.count(r)
#print(dic_rel)

dic_irr = {}

for i in palavras_irr:
    dic_irr[i]=palavras_irr.count(i)
#print(dic_irr)


dic_ambas = {**dic_rel,**dic_irr}
#print(dic_ambas)

print(len(dic_ambas))
    

1379


In [198]:
dic_prob_rel = {}

#print(dic_rel)

for r in dic_rel:
    dic_prob_rel[r] = (dic_rel[r] +1)/ (len(dic_rel) + len(dic_ambas))
    
print(dic_prob_rel)

dic_prob_irr = {}

for i in dic_irr:
    dic_prob_irr[i] = (dic_irr[i] +1)/ (len(dic_irr) + len(dic_ambas))
    
print(dic_prob_rel)


{'essa': 0.002631578947368421, 'budweiser': 0.07456140350877193, 'me': 0.005701754385964913, 'fud': 0.0008771929824561404, 'ontem,': 0.0008771929824561404, 'dor': 0.0021929824561403508, 'de': 0.03640350877192983, 'cabeça': 0.0021929824561403508, 'tá': 0.004824561403508772, 'tendo': 0.0008771929824561404, 'ms': 0.0013157894736842105, 'amigos': 0.0008771929824561404, '🤦🏿\u200d♂😂😂😂😂': 0.0008771929824561404, 'david3rick': 0.0021929824561403508, 'crlh': 0.0030701754385964912, 'com': 0.010526315789473684, 'red': 0.002631578947368421, 'bull': 0.002631578947368421, 'é': 0.016228070175438595, 'um': 0.010526315789473684, 'caminho': 0.002631578947368421, 'sem': 0.003947368421052632, 'volta': 0.0030701754385964912, 'deixa': 0.002631578947368421, 'mal': 0.004824561403508772, '\U0001f92d🍻🕺🏼': 0.002631578947368421, 'promodojaca': 0.0021929824561403508, 'jogo': 0.004824561403508772, 'do': 0.012719298245614035, 'mengão': 0.002631578947368421, 'amanhã': 0.005263157894736842, ',': 0.005263157894736842, '

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
